# **Implementing RAG with Langchain and Hugging Face** 
https://medium.com/international-school-of-ai-data-science/implementing-rag-with-langchain-and-hugging-face-28e3ea66c5f7

In [4]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-gpu

In [1]:
import langchain
langchain.__version__

'0.1.4'

## 1. Document Loading

In langchain a Document class has two attributes:
* page_content
* metadata

We use  hugging face dataset — [databricks-dolly-15k](https://huggingface.co/datasets/databricks/databricks-dolly-15k?row=3). This dataset is an open source dataset of instruction-following records generated by Databricks, including brainstorming, classification, closed QA, generation, information extraction, open QA, and summarization.


In [8]:
from langchain.document_loaders import HuggingFaceDatasetLoader

# Specify the dataset name and the column containing the content
dataset_name = "databricks/databricks-dolly-15k"
page_content_column = "context"  # or any other column you're interested

# Create a loader instance
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)

# Load the data
data = loader.load()

len(data) # 15011

# Display the first 5 entries
data[:5]

[Document(page_content='"Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia\'s domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney."', metadata={'instruction': 'When did Virgin Australia start operating?', 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}),
 Document(page_content='""', metadata={'instruction': 'Which is a species of fish? Tope or Rope', 'response': 'Tope', 'category': 'classification'}),
 Document(page_content='""', metadata={'instruction': 'Why can camels survive for long wit

In [3]:
print(data[0].page_content)
data[0].metadata

"Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney."


{'instruction': 'When did Virgin Australia start operating?',
 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.',
 'category': 'closed_qa'}

## 2. Document Transformers
Once the data is loaded, you can transform them to suit your application, or to fetch only the relevant parts of the document. Basically, it is about splitting a long document into smaller chunks which can fit your model and give results accurately and clearly.

There are several **“Text Splitters”** in LangChain, you have to choose according to your choice. **“RecursiveCharacterTextSplitter”** is recommended for generic text. It is parametrized by a list of characters. It tries to split the long texts recursively until the chunks are smaller enough.

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)

# len(docs) # 18502

docs[0]


Document(page_content='"Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia\'s domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney."', metadata={'instruction': 'When did Virgin Australia start operating?', 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'})

## 3. Text Embedding
* sentence-transformers/all-MiniLM-l6-v2 [$^{\textbf{[source]}}$](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) : This is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.

* BAAI/bge-large-en [$^{\textbf{[source]}}$](https://huggingface.co/BAAI/bge-large-en) : It's another embedding model that maps sentences & paragraphs to a 1024 dimensional dense vector space 

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

# Define the path to the pre-trained model you want to use
modelPath = "BAAI/bge-large-en"


# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs={'device':'cuda'}, # Pass the model configuration options
    encode_kwargs={'normalize_embeddings': False} # Pass the encoding options
)

# Example
len(embeddings.embed_query("This is a test document."))

1024

## Vector Stores
There is a need of databases so that we can store those embeddings and efficiently search them. Therefore, for storage and searching purpose, we need vector stores. You can retrieve the embedding vectors which will be “most similar”. Basically, it does a vector search for you. There are many vector stores integrated with LangChain, include`Pinecone`, `Weaviate`,`Chroma` and `Milvus`. We are using `Faiss` because it doesn’t require an API key.

In [9]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings) # take 4.5 minutes

In [10]:
# Example of similarity search for your question.
question = "What is cheesemaking?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)


"The goal of cheese making is to control the spoiling of milk into cheese. The milk is traditionally from a cow, goat, sheep or buffalo, although, in theory, cheese could be made from the milk of any mammal. Cow's milk is most commonly used worldwide. The cheesemaker's goal is a consistent product with specific characteristics (appearance, aroma, taste, texture). The process used to make a Camembert will be similar to, but not quite the same as, that used to make Cheddar.\n\nSome cheeses may be deliberately left to ferment from naturally airborne spores and bacteria; this approach generally leads to a less consistent product but one that is valuable in a niche market.\n\nCulturing\nCheese is made by bringing milk (possibly pasteurised) in the cheese vat to a temperature required to promote the growth of the bacteria that feed on lactose and thus ferment the lactose into lactic acid. These bacteria in the milk may be wild, as is the case with unpasteurised milk, added from a culture,


## Preparing the LLM Model
We'll use the model Dynamic-TinyBERT [$^{\textbf{[source]}}$](https://huggingface.co/Intel/dynamic_tinybert) : has been fine-tuned for the NLP task of question answering

In [ ]:
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline

# Specify the model name you want to use
model_name = "Intel/dynamic_tinybert"
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" # no suport for question-answering pepeline

# Load the tokenizer associated with the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

# Define a question-answering pipeline using the model and tokenizer
question_answerer = pipeline(
    "question-answering", 
    model=model_name, 
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)

## Retrievers
Once the data is in database, the LLM model is prepared, and the pipeline is created, we need to retrieve the data. A retriever is an interface that returns documents from the query.
 
**It is not able to store the documents**, only return or retrieves them. Basically, vector stores are the backbone of the retrievers. There are many retriever algorithms in LangChain.

In [16]:
# Create a retriever object from the 'db' with a search configuration 
# where it retrieves up to 4 relevant splits/documents.
# This retriever is likely used for retrieving data or documents from the database.
retriever = db.as_retriever(search_kwargs={"k": 4})

In [17]:
docs = retriever.get_relevant_documents("What is Cheesemaking?")
print(docs[0].page_content)

"The goal of cheese making is to control the spoiling of milk into cheese. The milk is traditionally from a cow, goat, sheep or buffalo, although, in theory, cheese could be made from the milk of any mammal. Cow's milk is most commonly used worldwide. The cheesemaker's goal is a consistent product with specific characteristics (appearance, aroma, taste, texture). The process used to make a Camembert will be similar to, but not quite the same as, that used to make Cheddar.\n\nSome cheeses may be deliberately left to ferment from naturally airborne spores and bacteria; this approach generally leads to a less consistent product but one that is valuable in a niche market.\n\nCulturing\nCheese is made by bringing milk (possibly pasteurised) in the cheese vat to a temperature required to promote the growth of the bacteria that feed on lactose and thus ferment the lactose into lactic acid. These bacteria in the milk may be wild, as is the case with unpasteurised milk, added from a culture,


## Retrieval QA Chain
The RetrievalQA chain, which combines question-answering with a retrieval step. To create it, we use a language model and a vector database as a retriever. By default, we put all the data together in a single batch, where the chain type is “stuff” when asking the language model. But if we have a lot of information and it doesn’t all fit at once, we can use methods like MapReduce, Refine, and MapRerank.



In [44]:
from langchain.llms import OpenAI

import openai
from dotenv import load_dotenv
import os

# load the OPENAI_API_KEY
load_dotenv('api_token.env')
openai.api_key = os.getenv("OPENAI_API_KEY")

# by default to using “text-davinci-003”.
llm = OpenAI(temperature=0.7)

In [45]:
from langchain.chains import RetrievalQA

# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine", # chain_type="stuff",
    retriever=retriever,
    return_source_documents=False)

In [46]:
question = "Who is Thomas Jefferson?"
result = qa.run({"query": question})
print(result["result"])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## Challenges with Open Source Models
When you will run your query, you will get the appropriate results, but with a ValueError, where it will ask you to input as a dictionary. While, in your final code, you have given your input as a dictionary. Doing same methods and processes with OpenAI api, you will not get any error like this.